### Google Drive

In [29]:
#https://stackoverflow.com/questions/47744131/colaboratory-can-i-access-to-my-google-drive-folder-and-file

# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse
# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# !mkdir -p drive
# !google-drive-ocamlfuse drive

### Parser

In [30]:
article=[]
reference=[]
summary =[]

import xml.etree.ElementTree
directory_path = 'drive/MyDrive/Colab Notebooks/Model 4/'
file_path = directory_path + 'result generator 5_1_2019.xml'

e = xml.etree.ElementTree.parse(file_path).getroot()

for atype in e.findall('example/article'):
    article.append(atype.text)

for atype in e.findall('example/reference'):
     reference.append(atype.text)

for atype in e.findall('example/summary'):
     summary.append(atype.text)

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### zaksum

In [32]:
!pip install sumeval
!python -m spacy download en

2023-12-13 13:20:33.905180: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-13 13:20:33.905239: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-13 13:20:33.905277: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-13 13:20:35.006205: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 96.8 MB/s eta 0:00:00
✔ Download and installation succe

In [33]:
import spacy
spacy.load('en_core_web_sm')

In [34]:
!pip install --upgrade sumeval
!pip install --upgrade sacrebleu


In [35]:
!pip3 install Rouge


In [45]:
pip install --upgrade sumeval


In [48]:
#https://github.com/chakki-works/sumeval
#https://github.com/Tian312/awesome-text-summarization

# from sumeval.metrics.bleu import BLEUCalculator
# !pip3 install rouge-score


# from xml.etree.ElementTree import Element, SubElement, Comment, tostring
# from xml.dom import minidom
# from functools import reduce
# from sumeval.metrics.rouge import RougeCalculator
# import sacrebleu
# from rouge_score import rouge_scorer

from sumeval.metrics.rouge import RougeCalculator
import sacrebleu
from rouge_score import rouge_scorer



def eval_rouges(reference_summary, model_summary):
    try:
        rouge = RougeCalculator(stopwords=True, lang="en")

        rouge_1 = rouge.rouge_n(
            summary=model_summary,
            references=[reference_summary],
            n=1
        )
        rouge_2 = rouge.rouge_n(
            summary=model_summary,
            references=[reference_summary],
            n=2
        )
        rouge_l = rouge.rouge_l(
            summary=model_summary,
            references=[reference_summary]
        )

        # BLEU calculation using sacrebleu
        bleu = sacrebleu.corpus_bleu([model_summary], [[reference_summary]])
        bleu_score = bleu.score

        return rouge_1, rouge_2, rouge_l, None, bleu_score  # No ROUGE-BE score
    except Exception as e:
        print(f"Error in eval_rouges: {e}")
        return None, None, None, None, None




# def eval_rouges(reference_summary, model_summary):
#     try:
#         rouge = RougeCalculator(stopwords=True, lang="en")

#         rouge_1 = rouge.rouge_n(
#             summary=model_summary,
#             references=[reference_summary],
#             n=1
#         )
#         rouge_2 = rouge.rouge_n(
#             summary=model_summary,
#             references=[reference_summary],
#             n=2
#         )
#         rouge_l = rouge.rouge_l(
#             summary=model_summary,
#             references=[reference_summary]
#         )

#         # ROUGE-BE requires a spaCy model for tokenization
#         rouge_be = rouge_scorer.RougeScorer(['rougeBE'], use_stemmer=True)
#         rouge_be_score = rouge_be.score(reference_summary, model_summary)['rougeBE']

#         # BLEU calculation using sacrebleu
#         bleu = sacrebleu.corpus_bleu([model_summary], [[reference_summary]])
#         bleu_score = bleu.score

#         return rouge_1, rouge_2, rouge_l, rouge_be_score, bleu_score
#     except Exception as e:
#         print(f"Error in eval_rouges: {e}")
#         return None, None, None, None, None


In [37]:
#https://pymotw.com/2/xml/etree/ElementTree/create.html

bleu_arr = []
rouge_1_arr  = []
rouge_2_arr  = []
rouge_L_arr  = []
rouge_be_arr = []

from xml.etree import ElementTree
from xml.dom import minidom
from functools import reduce

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

from xml.etree.ElementTree import Element, SubElement, Comment

top = Element('ZakSum')

def zaksum(article, reference, summary_array, default_path):
    import spacy

    nlp = spacy.load("en_core_web_sm")

    top = Element('top')

    comment = Comment('Generated by Amr Zaki')
    top.append(comment)

    i = 0
    bleu_arr, rouge_1_arr, rouge_2_arr, rouge_L_arr, rouge_be_arr = [], [], [], [], []
    for summ in summary_array:
        example = SubElement(top, 'example')
        article_element = SubElement(example, 'article')
        article_element.text = article[i]

        reference_element = SubElement(example, 'reference')
        reference_element.text = reference[i]

        summary_element = SubElement(example, 'summary')
        summary_element.text = summ

        rouge_1, rouge_2, rouge_L, rouge_be, bleu_score = eval_rouges(reference[i], summ)

        eval_element = SubElement(example, 'eval')
        bleu_score_element = SubElement(eval_element, 'BLEU', {'score': str(bleu_score)})
        ROUGE_1_element = SubElement(eval_element, 'ROUGE_1', {'score': str(rouge_1)})
        ROUGE_2_element = SubElement(eval_element, 'ROUGE_2', {'score': str(rouge_2)})
        ROUGE_L_element = SubElement(eval_element, 'ROUGE_l', {'score': str(rouge_L)})
        ROUGE_be_element = SubElement(eval_element, 'ROUGE_be', {'score': str(rouge_be)})

        bleu_arr.append(bleu_score)
        rouge_1_arr.append(rouge_1)
        rouge_2_arr.append(rouge_2)
        rouge_L_arr.append(rouge_L)
        rouge_be_arr.append(rouge_be)

        i += 1

    # Handling division by zero for empty arrays
    bleu_average = reduce(lambda x, y: x + y if y is not None else x, bleu_arr, 0) / len(bleu_arr) if len(bleu_arr) > 0 else 0
    rouge_1_average = reduce(lambda x, y: x + y if y is not None else x, rouge_1_arr, 0) / len(rouge_1_arr) if len(rouge_1_arr) > 0 else 0
    rouge_2_average = reduce(lambda x, y: x + y if y is not None else x, rouge_2_arr, 0) / len(rouge_2_arr) if len(rouge_2_arr) > 0 else 0
    rouge_L_average = reduce(lambda x, y: x + y if y is not None else x, rouge_L_arr, 0) / len(rouge_L_arr) if len(rouge_L_arr) > 0 else 0
    rouge_be_average = reduce(lambda x, y: x + y if y is not None else x, rouge_be_arr, 0) / len(rouge_be_arr) if len(rouge_be_arr) > 0 else 0

    top.set('bleu', str(bleu_average))
    top.set('rouge_1', str(rouge_1_average))
    top.set('rouge_2', str(rouge_2_average))
    top.set('rouge_L', str(rouge_L_average))
    top.set('rouge_be', str(rouge_be_average))

    with open(default_path + "result.xml", "w+") as f:
        print(minidom.parseString(tostring(top)).toprettyxml(indent="  "), file=f)


# def zaksum(article, reference, summary_array, default_path):
#     import spacy
#     from xml.etree.ElementTree import Element, SubElement, Comment, tostring
#     from xml.dom import minidom
#     from functools import reduce

#     # def eval_rouges(reference, hypothesis):
#     #     # Add your implementation of ROUGE and BLEU calculation here
#     #     pass

#     # top = Element('top')

#     # nlp = spacy.load("en_core_web_sm")

#     # comment = Comment('Generated by Amr Zaki')
#     # top.append(comment)

#     # i = 0
#     # bleu_arr, rouge_1_arr, rouge_2_arr, rouge_L_arr, rouge_be_arr = [], [], [], [], []
#     # for summ in summary_array:
#     #     example = SubElement(top, 'example')
#     #     article_element = SubElement(example, 'article')
#     #     article_element.text = article[i]

#     #     reference_element = SubElement(example, 'reference')
#     #     reference_element.text = reference[i]

#     #     summary_element = SubElement(example, 'summary')
#     #     summary_element.text = summ

#     #     rouge_1, rouge_2, rouge_L, rouge_be, bleu_score = eval_rouges(reference[i], summ)

#     #     eval_element = SubElement(example, 'eval')
#     #     bleu_score_element = SubElement(eval_element, 'BLEU', {'score': str(bleu_score)})
#     #     ROUGE_1_element = SubElement(eval_element, 'ROUGE_1', {'score': str(rouge_1)})
#     #     ROUGE_2_element = SubElement(eval_element, 'ROUGE_2', {'score': str(rouge_2)})
#     #     ROUGE_L_element = SubElement(eval_element, 'ROUGE_l', {'score': str(rouge_L)})
#     #     ROUGE_be_element = SubElement(eval_element, 'ROUGE_be', {'score': str(rouge_be)})

#     #     bleu_arr.append(bleu_score)
#     #     rouge_1_arr.append(rouge_1)
#     #     rouge_2_arr.append(rouge_2)
#     #     rouge_L_arr.append(rouge_L)
#     #     rouge_be_arr.append(rouge_be)

#     #     i += 1

#     # top.set('bleu', str(reduce(lambda x, y: x + y, bleu_arr) / len(bleu_arr)))
#     # top.set('rouge_1', str(reduce(lambda x, y: x + y, rouge_1_arr) / len(rouge_1_arr)))
#     # top.set('rouge_2', str(reduce(lambda x, y: x + y, rouge_2_arr) / len(rouge_2_arr)))
#     # top.set('rouge_L', str(reduce(lambda x, y: x + y, rouge_L_arr) / len(rouge_L_arr)))
#     # top.set('rouge_be', str(reduce(lambda x, y: x + y, rouge_be_arr) / len(rouge_be_arr)))

#     # with open(default_path + "result.xml", "w+") as f:
#     #     print(minidom.parseString(tostring(top)).toprettyxml(indent="  "), file=f)


#Run

In [49]:
zaksum(article,reference,summary,"drive/MyDrive/Colab Notebooks/Model 4/")